In [28]:
import pandas as pd
import os

In [29]:
df = pd.read_csv(r'D:\Insurance Claim Prediction Project\notebook\data\processed\processed_data.csv')
df.head()

,age,policy_state,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,auto_make,auto_model,auto_year,fraud_reported,policy_bind_year,policy_bind_month,policy_bind_day,incident_date_year,incident_date_month,incident_date_day,csl_per_person
0,48,OH,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,5,1,YES,1,2,YES,71610,Saab,92x,2004,Y,2014,10,17,2015,1,25,250
1,42,IN,2000,1197.22,5000000,MALE,MD,machine-op-inspct,reading,other-relative,0,0,Vehicle Theft,No Collision,Minor Damage,Police,VA,Riverwood,8,1,?,0,0,NO,5070,Mercedes,E400,2007,Y,2006,6,27,2015,1,21,250
2,29,OH,2000,1413.14,5000000,FEMALE,PhD,sales,board-games,own-child,35100,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7,3,NO,2,3,NO,34650,Dodge,RAM,2007,N,2000,9,6,2015,2,22,100
3,41,IL,2000,1415.74,6000000,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,5,1,?,1,2,NO,63400,Chevrolet,Tahoe,2014,Y,1990,5,25,2015,1,10,250
4,44,IL,1000,1583.91,6000000,MALE,Associate,sales,board-games,unmarried,66000,-46000,Vehicle Theft,No Collision,Minor Damage,Police,NY,Arlington,20,1,NO,0,1,NO,6500,Accura,RSX,2009,N,2014,6,6,2015,2,17,500


In [30]:
X,y = df.drop('fraud_reported',axis=1),df['fraud_reported']

In [31]:
y=y.map({'Y': 1,'N': 0})

In [32]:
## train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [33]:
## Importing preprocessor and handling imbalance data
import pickle
from imblearn.over_sampling import SMOTE
preprocessor_file_path=os.path.join(os.getcwd(),'preprocessor.pkl')

with open(preprocessor_file_path,'rb') as file_obj:
    preprocessor=pickle.load(file_obj)

In [34]:
## performing  standardization 
X_train = preprocessor.fit_transform(X_train,y_train)
X_test = preprocessor.transform(X_test)
X_test

array([[-1.4641271 ,  1.3913488 , -0.53921609, ...,  0.19491767,
         0.16570152,  0.22524942],
       [-0.70026181, -0.23053989,  0.74079248, ...,  0.22826856,
         0.13586366,  0.16332441],
       [ 1.26396322, -0.23053989, -1.25881778, ...,  0.22826856,
         0.22088077,  0.22524942],
       ...,
       [-0.37289097, -0.23053989, -1.37944569, ...,  0.19491767,
         0.34754026,  0.24917852],
       [-0.26376736, -0.23053989, -0.5391713 , ...,  0.26469871,
         0.16570152,  0.22524942],
       [ 1.1548396 , -0.23053989,  0.08058649, ...,  0.22826856,
         0.16570152,  0.22524942]], shape=(300, 45))

In [35]:
## importing smote
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=42)
X_train,y_train=smote.fit_resample(X_train,y_train)

In [39]:
## training the model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

classification_models = {
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "CatBoost": CatBoostClassifier(verbose=False),
    "Naive Bayes (Gaussian)": GaussianNB()
}


For fraud detection, recall is usually the most critical, because missing a fraudulent claim costs money, while a few false positives (extra checks) are manageable. So we focus mainly on recall

In [40]:
from sklearn.metrics import recall_score,roc_auc_score,precision_score,accuracy_score,f1_score
def evaluate_model(y_true,y_pred):
    model_recall_score = recall_score(y_true,y_pred)
    model_roc_auc_score = roc_auc_score(y_true,y_pred)
    model_precision_score = precision_score(y_true,y_pred)
    model_accuracy_score = accuracy_score(y_true,y_pred)
    model_f1_score = f1_score(y_true,y_pred)
    return model_recall_score,model_roc_auc_score,model_precision_score,model_accuracy_score,model_f1_score
    

In [41]:

for i in range(len(list(classification_models))):
    model = list(classification_models.values())[i]
    model.fit(X_train,y_train)
    
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    
    y_train_recall_score,y_train_roc_auc_score,y_train_precision_score,y_train_accuracy_score,y_train_f1_score=(evaluate_model(y_train,y_train_pred))
    y_test_recall_score,y_test_roc_auc_score,y_test_precision_score,y_test_accuracy_score,y_test_f1_score=(evaluate_model(y_test,y_test_pred))
    
    print(list(classification_models.keys())[i])
    
    print('Model performance for Training set')
    print("- Recall Score: {:.4f}".format(y_train_recall_score))
    print("- Roc Auc Score: {:.4f}".format(y_train_roc_auc_score))
    print("- Precision Score: {:.4f}".format(y_train_precision_score))
    print("- Accuracy Score: {:.4f}".format(y_train_accuracy_score))
    print("- Score: {:.4f}".format(y_train_f1_score))
    print('----------------------------------')
    
    print('Model performance for Test set')
    print('Model performance for Training set')
    print("- Recall Score: {:.4f}".format(y_test_recall_score))
    print("- Roc Auc Score: {:.4f}".format(y_test_roc_auc_score))
    print("- Precision Score: {:.4f}".format(y_test_precision_score))
    print("- Accuracy Score: {:.4f}".format(y_test_accuracy_score))
    print("- F1 Score: {:.4f}".format(y_test_f1_score))
    
    print('='*35)
    print('\n')

Logistic Regression
Model performance for Training set
- Recall Score: 0.9174
- Roc Auc Score: 0.8884
- Precision Score: 0.8670
- Accuracy Score: 0.8884
- Score: 0.8915
----------------------------------
Model performance for Test set
Model performance for Training set
- Recall Score: 0.7500
- Roc Auc Score: 0.7932
- Precision Score: 0.6250
- Accuracy Score: 0.8133
- F1 Score: 0.6818


K-Nearest Neighbors
Model performance for Training set
- Recall Score: 0.9850
- Roc Auc Score: 0.7702
- Precision Score: 0.6890
- Accuracy Score: 0.7702
- Score: 0.8108
----------------------------------
Model performance for Test set
Model performance for Training set
- Recall Score: 0.7875
- Roc Auc Score: 0.6256
- Precision Score: 0.3481
- Accuracy Score: 0.5500
- F1 Score: 0.4828


Decision Tree
Model performance for Training set
- Recall Score: 1.0000
- Roc Auc Score: 1.0000
- Precision Score: 1.0000
- Accuracy Score: 1.0000
- Score: 1.0000
----------------------------------
Model performance for Te

In [49]:
param_grids = {
    "Logistic Regression": {
        "penalty": ["l1", "l2", "elasticnet", "none"],
        "C": [0.01, 0.1, 1, 10, 100],
        "solver": ["lbfgs", "liblinear", "saga", "newton-cg"],
        "max_iter": [100, 200, 500],
        "class_weight": [None, "balanced"]
    },
    
    "K-Nearest Neighbors": {
        "n_neighbors": [3, 5, 7, 9, 11],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan", "minkowski"],
        "p": [1, 2]
    },
    
    "Decision Tree": {
        "criterion": ["gini", "entropy"],
        "max_depth": [None, 5, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": [None, "sqrt", "log2"],
        "class_weight": [None, "balanced"]
    },
    
    "Random Forest": {
        "n_estimators": [100, 200, 300],
        "criterion": ["gini", "entropy"],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2", None],
        "bootstrap": [True, False],
        "class_weight": [None, "balanced"]
    },
    
    "Extra Trees": {
        "n_estimators": [100, 200, 300],
        "criterion": ["gini", "entropy"],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2", None],
        "bootstrap": [True, False],
        "class_weight": [None, "balanced"]
    },
    
    "AdaBoost": {
        "n_estimators": [50, 100, 200, 300],
        "learning_rate": [0.001, 0.01, 0.1, 0.5, 1],
        "algorithm": ["SAMME", "SAMME.R"]
    },
    
    "Gradient Boosting": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.001, 0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "subsample": [0.6, 0.8, 1.0],
        "max_features": [None, "sqrt", "log2"]
    },
    
    "XGBoost": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.001, 0.01, 0.1, 0.2],
        "max_depth": [3, 5, 7, 10],
        "subsample": [0.6, 0.8, 1.0],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "gamma": [0, 0.1, 0.3, 0.5],
        "reg_alpha": [0, 0.01, 0.1, 1],
        "reg_lambda": [1, 1.5, 2]
    },
    
    "CatBoost": {
        "depth": [4, 6, 8, 10, 12],
        "learning_rate": [0.01, 0.03, 0.05, 0.1, 0.2],
        "iterations": [200, 500, 800, 1000],
        "l2_leaf_reg": [1, 3, 5, 7, 9],
        "bagging_temperature": [0, 0.5, 1, 2, 5],
        "random_strength": [0, 1, 2, 5, 10],
        "rsm": [0.6, 0.8, 1.0],
        "grow_policy": ["SymmetricTree", "Depthwise", "Lossguide"],
        "min_data_in_leaf": [1, 5, 10, 20, 50],
        "max_bin": [128, 254, 512]
    },
    
    "Naive Bayes (Gaussian)": {
        "var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6]
    }
}


In [50]:
# Hyper Parameter Tuning
from sklearn.model_selection import RandomizedSearchCV
best_models={}
for name, model in classification_models.items():
    print(f"Tuning {name}...")
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grids[name],
        n_iter=20,              # number of combinations to try
        cv=5,                   # 5-fold cross-validation
        scoring="recall",
        n_jobs=-1,
        random_state=42,
        verbose=False
    )
    search.fit(X_train, y_train)
    best_models[name] = search.best_estimator_
    print(f"Best Params for {name}: {search.best_params_}")
    print(f"Best CV Recall Score: {search.best_score_}")
    print("-"*50)


Tuning Logistic Regression...


d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
70 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1218

Best Params for Logistic Regression: {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 500, 'class_weight': 'balanced', 'C': 1}
Best CV R²: 0.9043731264327279
--------------------------------------------------
Tuning K-Nearest Neighbors...
Best Params for K-Nearest Neighbors: {'weights': 'distance', 'p': 1, 'n_neighbors': 9, 'metric': 'euclidean'}
Best CV R²: 0.9849761946746606
--------------------------------------------------
Tuning Decision Tree...
Best Params for Decision Tree: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 5, 'criterion': 'gini', 'class_weight': 'balanced'}
Best CV R²: 0.9119202962440486
--------------------------------------------------
Tuning Random Forest...
Best Params for Random Forest: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 30, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': True}
Best CV R²: 0.9475401163815906
---------------------------

d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
45 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
  File "d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\base.py", line 471, in _validate_params
    validate_p

Best Params for AdaBoost: {'n_estimators': 200, 'learning_rate': 0.01, 'algorithm': 'SAMME'}
Best CV R²: 0.9569035443484394
--------------------------------------------------
Tuning Gradient Boosting...
Best Params for Gradient Boosting: {'subsample': 0.6, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 7, 'learning_rate': 0.01}
Best CV R²: 0.9418973725974255
--------------------------------------------------
Tuning XGBoost...
Best Params for XGBoost: {'subsample': 1.0, 'reg_lambda': 1.5, 'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 1.0}
Best CV R²: 0.95316522659143
--------------------------------------------------
Tuning CatBoost...
Best Params for CatBoost: {'rsm': 0.8, 'random_strength': 2, 'min_data_in_leaf': 50, 'max_bin': 128, 'learning_rate': 0.03, 'l2_leaf_reg': 5, 'iterations': 500, 'grow_policy': 'SymmetricTree', 'depth': 12, 'bagging_temperature': 0}
Best 

d:\Insurance Claim Prediction Project\venv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=20. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


## Out of all these models CatBosst Performs the best with the recall score -  0.9662669723152882

In [ ]:
best_params={'rsm': 0.8, 
             'random_strength': 2, 
             'min_data_in_leaf': 50, 
             'max_bin': 128, 
             'learning_rate': 0.03, 
             'l2_leaf_reg': 5, 
             'iterations': 500, 
             'grow_policy': 'SymmetricTree', 
             'depth': 12, 
             'bagging_temperature': 0}


In [70]:
best_model = CatBoostClassifier(**best_params,verbose=False,early_stopping_rounds=50,class_weights=[1, 5])
best_model.fit(X_train,y_train)
y_test_pred=best_model.predict(X_test)
print(f'Recall Score {recall_score(y_test,y_test_pred)}')
print(f'Roc_Auc_Score {roc_auc_score(y_test,y_test_pred)}')


Recall Score 0.825
Roc_Auc_Score 0.8284090909090909


In [73]:
import pickle
model_file_path=os.path.join(os.getcwd(),'model.pkl')
with open(model_file_path,'wb') as file_obj:
    pickle.dump(best_models,file_obj)
